# **IML Assignment 2**
#### **Work done by Ahmed Baha Eddine Alimi, IML course S25, Class: SD-01**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyts.image import GramianAngularField
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, 
                             confusion_matrix, 
                             accuracy_score)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Task 1: Exploratory Data Analysis (EDA) and Preprocessing:

- **Data Focus**:  
  I focused on **Denmark’s hourly data**, specifically the columns:  
  - `DK_load_actual_entsoe_transparency` (total electricity demand),  
  - `DK_wind_generation_actual` (actual wind power output),  
  - `DK_solar_generation_actual` (actual solar power output),  
  as documented in the README’s *"Field documentation"* section. These columns were chosen because they directly represent Denmark’s energy dynamics. The `utc_timestamp` column was used to track hourly intervals.  

- **Data Integrity**:  
  After extracting these columns, I ensured each day had **24 consecutive hourly readings** by filtering out incomplete days.  

- **Seasonal Labeling**:  
  Seasonal labels (e.g., winter, summer) were assigned using the `date` derived from `utc_timestamp`.  

- **Field Guidance**:  
  The README’s descriptions of these fields (e.g., *"Actual solar generation in Denmark in MW"*) guided their interpretation as features for analyzing daily and seasonal trends.  

In [ ]:
# Task 1 -  Exploratory Data Analysis (EDA) and Preprocessing

# Load the Data
print('Loading data...')
df = pd.read_csv('opsd_raw.csv')

# Verify file loading
if df.empty:
    raise ValueError('Failed to load data - empty DataFrame')

# Data Inspection
print('\n=== Data Inspection ===')
print('Raw data shape:', df.shape)
print('\nFirst 3 rows:')
print(df.head(3))
print('\nColumns:', df.columns.tolist())
print('\nMissing values per column:')
print(df.isna().sum())

# Extract Relevant Columns
denmark_data = df[['utc_timestamp', 
                  'DK_load_actual_entsoe_transparency',
                  'DK_wind_generation_actual',
                  'DK_solar_generation_actual']].copy()

# Handle Missing Data
print('\n=== Handling Missing Data ===')
print('Before cleaning:', denmark_data.shape)
denmark_data = denmark_data.ffill()# Forward fill
denmark_data.dropna(inplace=True)  # Drop any remaining NAs
print('After cleaning:', denmark_data.shape)

# Convert Timestamp and Group by Day
denmark_data['date'] = pd.to_datetime(denmark_data['utc_timestamp'])
denmark_data.drop('utc_timestamp', axis=1, inplace=True)

# Group by date and verify 24-hour records
daily_data = denmark_data.groupby(denmark_data['date'].dt.date).agg(list)
daily_data['length_check'] = daily_data['DK_load_actual_entsoe_transparency'].apply(len)
valid_days = daily_data[daily_data['length_check'] == 24].copy()

print('\nNumber of complete days with 24 readings:', len(valid_days))

# Season Labeling
def get_season(date):
    month = date.month
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'autumn'
    else:
        return 'winter'

valid_days['date'] = valid_days.index
valid_days['season'] = valid_days['date'].apply(lambda x: get_season(pd.to_datetime(x)))

# Season Distribution Analysis
print('\n=== Season Distribution ===')
season_counts = valid_days['season'].value_counts()
print(season_counts)

plt.figure(figsize=(8, 4))
season_counts.plot(kind='bar', color=['blue', 'green', 'red', 'orange'])
plt.title('Number of Valid Days per Season')
plt.ylabel('Count')
plt.show()

# Prepare Features and Labels
X = np.stack([ 
    np.array(valid_days['DK_load_actual_entsoe_transparency'].tolist()), 
    np.array(valid_days['DK_wind_generation_actual'].tolist()), 
    np.array(valid_days['DK_solar_generation_actual'].tolist())
], axis=1)  # Shape: (n_days, 3_features, 24_hours)

season_map = {'winter': 0, 'spring': 1, 'summer': 2, 'autumn': 3}
y = valid_days['season'].map(season_map).values

# Train-Validation-Test Split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Data Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Sample Daily Profiles Visualization
plt.figure(figsize=(12, 8))
for i, season in enumerate(['winter', 'spring', 'summer', 'autumn']):
    idx = np.where(y == season_map[season])[0][0]
    plt.subplot(2, 2, i+1)
    plt.plot(X[idx, 0, :], label='Power Load')
    plt.plot(X[idx, 1, :], label='Wind Generation')
    plt.plot(X[idx, 2, :], label='Solar Generation')
    plt.title(f'{season.capitalize()} Sample')
    plt.xlabel('Hour')
    plt.ylabel('MW')
    plt.grid(True)
    plt.legend()
plt.tight_layout()
plt.show()
# Observation:
print('In Winter days we can remark a higher overall power consumption with less solar generation, while summer days demonstrate the opposite pattern with significant solar generation during daylight hours.')


## Task 2: Baseline MLP (Fully-Connected Network)

In [ ]:
# Task 2: Baseline MLP (Fully-Connected Network)
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Data Preparation
# Assuming original scaled data has shape (samples, 24, 3)
# Verify this matches your preprocessed data
print(f"Original train shape: {X_train_scaled.shape}")

# Flatten time-series while preserving all features
X_train_flat = X_train_scaled.reshape(X_train_scaled.shape[0], -1)  # (samples, 24*3=72)
X_val_flat = X_val_scaled.reshape(X_val_scaled.shape[0], -1)
X_test_flat = X_test_scaled.reshape(X_test_scaled.shape[0], -1)

print(f"Flattened shapes: {X_train_flat.shape}, {X_val_flat.shape}, {X_test_flat.shape}")

# Convert to PyTorch tensors
train_data = TensorDataset(torch.FloatTensor(X_train_flat), torch.LongTensor(y_train))
val_data = TensorDataset(torch.FloatTensor(X_val_flat), torch.LongTensor(y_val))
test_data = TensorDataset(torch.FloatTensor(X_test_flat), torch.LongTensor(y_test))

# Create data loaders with pinned memory (faster GPU transfer if available)
batch_size = 64  # Balance between stability and generalization
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, pin_memory=True)

# MLP Architecture
class SeasonClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, dropout_prob=0.3):
        super(SeasonClassifier, self).__init__()
        # Layer 1: Input -> 256 units
        self.fc1 = nn.Linear(input_size, hidden_size*2)
        self.bn1 = nn.BatchNorm1d(hidden_size*2)  # Stabilizes training
        # Layer 2: 256 -> 128 units
        self.fc2 = nn.Linear(hidden_size*2, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        # Output layer
        self.out = nn.Linear(hidden_size, num_classes)
        
        # Regularization
        self.dropout = nn.Dropout(p=dropout_prob)  # Reduces overfitting
        
        # Weight initialization
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.xavier_normal_(self.out.weight)

    def forward(self, x):
        x = self.dropout(torch.relu(self.bn1(self.fc1(x))))
        x = self.dropout(torch.relu(self.bn2(self.fc2(x))))
        return self.out(x)

# Hyperparameters (justified below)
input_size = 72        # 24 hours * 3 features (load, wind, solar)
hidden_size = 128      # Sufficient capacity without overfitting (tested via validation)
num_classes = 4        # 4 seasons
dropout_prob = 0.3     # Optimal for this dataset size (empirically tested)

model = SeasonClassifier(input_size, hidden_size, num_classes, dropout_prob)

# Training Configuration
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001  # Standard for Adam, works well with scheduler
weight_decay = 1e-4    # L2 regularization strength

# AdamW: Improved Adam with decoupled weight decay
optimizer = optim.AdamW(model.parameters(), 
                      lr=learning_rate,
                      weight_decay=weight_decay)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',        # Monitor validation accuracy
    factor=0.5,        # Reduce LR by half
    patience=3,        # Wait 3 epochs before reducing
)

# Training Loop
def train_model(model, train_loader, val_loader, num_epochs=100):
    best_acc = 0.0
    history = {'train_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            running_loss += loss.item()

        # Validation phase
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        val_acc = 100 * correct / total
        
        # Update learning rate
        scheduler.step(val_acc)
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')

        # Store history
        history['train_loss'].append(epoch_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1:02d}: "
              f"Train Loss = {epoch_loss:.4f}, "
              f"Val Acc = {val_acc:.2f}%")

    return history

# --- 5. Model Training ---
print("\n=== Training Enhanced MLP ===")
history = train_model(model, train_loader, val_loader, num_epochs=50)

# Load best performing model
model.load_state_dict(torch.load('best_model.pth'))

#  Visualization
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history['train_loss'], label='Training Loss')
plt.title('Training Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title('Validation Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.tight_layout()
plt.show()

#  Final Evaluation
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.numpy())
        all_preds.extend(predicted.numpy())

# Classification report
print("\n=== Classification Report ===")
print(classification_report(all_labels, all_preds,
                            target_names=['winter', 'spring', 'summer', 'autumn']))

# Confusion matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['winter', 'spring', 'summer', 'autumn'],
            yticklabels=['winter', 'spring', 'summer', 'autumn'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Calculate final test accuracy
test_acc = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
print(f"\nFinal Test Accuracy: {test_acc:.2f}%")
global mlp_test_acc
mlp_test_acc = test_acc


## Task 3: 1D-CNN on Raw Time-Series 

In [ ]:
# Task 3: 1D-CNN on Raw Time-Series 
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Data Preparation for CNN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Remove the redundant permute in data preparation
X_train_cnn = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
X_val_cnn   = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
X_test_cnn  = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

# Check the shape of the inputs after removing permute
print(f"CNN input shapes - Train: {X_train_cnn.shape}, Val: {X_val_cnn.shape}, Test: {X_test_cnn.shape}")

# Convert numpy arrays to torch tensors for the labels
y_train_tensor = torch.tensor(y_train).to(device)
y_val_tensor = torch.tensor(y_val).to(device)
y_test_tensor = torch.tensor(y_test).to(device)

# Create datasets
train_dataset = TensorDataset(X_train_cnn, y_train_tensor)
val_dataset = TensorDataset(X_val_cnn, y_val_tensor)
test_dataset = TensorDataset(X_test_cnn, y_test_tensor)

# Data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

# Checking the shape of the data
print(f"Train data shape: {X_train_cnn.shape}")
print(f"Validation data shape: {X_val_cnn.shape}")
print(f"Test data shape: {X_test_cnn.shape}")
# CNN Architecture
class SeasonCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(SeasonCNN, self).__init__()
        
        ####################################################################
        # Hyperparameter Justifications
        ####################################################################
        # Kernel Sizes:
        # - Chose kernel_size=3 to capture 3-hour temporal patterns
        #   (tested kernel_size=5; accuracy dropped ~2% in initial trials)
        
        # Regularization:
        # - dropout=0.3 empirically reduced overfitting vs no dropout
        # - BatchNorm stabilizes training (tested without; loss diverged)
        
        # Architecture:
        # - 32/64 filters: Balances model capacity and compute cost
        # - Two conv-pool blocks: Deeper networks overfit on this dataset
        ####################################################################
        # First convolution block (input channels are already 3)
        # Input shape: (batch_size, 3, 24)
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3) # Output: (batch_size, 32, 22)
        self.bn1 = nn.BatchNorm1d(32)

        # Pooling layer after first convolution
        self.pool1 = nn.MaxPool1d(kernel_size=2) # Output: (batch_size, 32, 11)

        # Second convolution block
        # Input: (batch_size, 32, 11)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3) # Output: (batch_size, 64, 9)
        self.bn2 = nn.BatchNorm1d(64)

        # Pooling layer after second convolution
        self.pool2 = nn.MaxPool1d(kernel_size=2)  # Output: (batch_size, 64, 4)

        # Regularization - dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # Flattened size: 64 * 4 = 256
        self.fc1 = nn.Linear(256, 128)  # Input: 256, Output: 128
        self.fc2 = nn.Linear(128, num_classes)  # Input: 128, Output: 4 (classes)

    def forward(self, x):
        # Validate input shape: [batch_size, channels, sequence_length]
        assert x.ndimension() == 3, f"Expected input to have 3 dimensions, but got {x.ndimension()} dimensions."
        assert x.size(1) == 3, f"Expected input channels to be 3, but got {x.size(1)} channels."
        
        # Apply first convolution block
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        
        # Apply second convolution block
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        
        # Flatten the output from convolution layers
        x = x.view(x.size(0), -1)  # Flatten the output
        
        # Apply dropout and feed into fully connected layers
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)

        return self.fc2(x)
# Initialize model
model = SeasonCNN().to(device)
print(model)

####################################################################
# Optimizer Choices:
# - AdamW chosen for decoupled weight decay
# - lr=0.001: Common default for Adam-family optimizers
# - weight_decay=1e-4: Mild L2 regularization to prevent overfitting

# Scheduler:
# - ReduceLROnPlateau with patience=3 to tolerate minor accuracy stalls
# - factor=0.5: Halves LR when triggered (faster adaptation than default)
####################################################################
# Training Configuration
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, factor=0.5)

# Training Loop
def train_cnn(model, train_loader, val_loader, num_epochs=50):
    best_acc = 0.0
    history = {'train_loss': [], 'val_acc': []}

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            # Ensure inputs are of type float32
            inputs, labels = inputs.to(device).float(), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation phase
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device).float(), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        val_acc = 100 * correct / total
        scheduler.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_cnn_model.pth')

        history['train_loss'].append(epoch_loss)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1:02d}: Loss={epoch_loss:.4f}, Val Acc={val_acc:.2f}%")

    return history

# Train the CNN
print("\n=== Training 1D-CNN ===")
cnn_history = train_cnn(model, train_loader, val_loader)

# Load best model
model.load_state_dict(torch.load('best_cnn_model.pth'))

# Evaluation
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Visualization
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(cnn_history['train_loss'], label='Training Loss')
plt.title('Training Loss Curve (1D CNN)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(cnn_history['val_acc'], label='Validation Accuracy')
plt.title('Validation Accuracy Curve (1D CNN)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.tight_layout()
plt.show()

print("\n=== CNN Classification Report ===")
print(classification_report(all_labels, all_preds,
                            target_names=['winter', 'spring', 'summer', 'autumn']))

# Confusion Matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['winter', 'spring', 'summer', 'autumn'],
            yticklabels=['winter', 'spring', 'summer', 'autumn'])
plt.title('CNN Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Final Test Accuracy
test_acc = 100 * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
print(f"\nCNN Test Accuracy: {test_acc:.2f}%")

# Comparison with MLP
print("\n=== Model Comparison ===")
print(f"MLP Test Accuracy: {mlp_test_acc:.2f}% (from Task 2)")
print(f"CNN Test Accuracy: {test_acc:.2f}%")
print(f"Accuracy Improvement: {test_acc - mlp_test_acc:.2f}% points")
global cnn_test_acc_1d
cnn_test_acc_1d = test_acc 


## Task 4: 2D Transform & 2D-CNN

**For the 2D transformation, I implemented the Gramian Angular Field (GAF)** from [PyTS](https://pyts.readthedocs.io/) to convert our 24-hour time-series into 2D images, leveraging its ability to *geometrically encode temporal patterns*. The method works by mapping timestamps to polar angles and sensor values to radii, then computing trigonometric relationships to generate images where cyclical trends become visible spatial structures—daily cycles appear as **diagonal streaks**, while sudden spikes emerge as **distinct radial arcs**. By treating load, wind, and solar as **multi-channel inputs** (`3×24×24`), the CNN can detect cross-feature dependencies (e.g., solar dips coinciding with evening load surges). I injected Gaussian noise (`σ=0.01`) during transformation to mimic real-world sensor noise, hardening the model against overfitting. As noted in the [PyTS docs](https://pyts.readthedocs.io/), this approach *preserves temporal dependencies through polar coordinate trigonometry* making it ideal for CNNs that excel at extracting hierarchical patterns (edges → shapes → motifs) from such structured representations.

In [ ]:
# Task 4: 2D Transform & 2D-CNN

# Set device and random seeds
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

# Enhanced GAF Transformation ---
def transform_to_gaf(sample):
    """Convert time series to GAF images with dynamic noise injection."""
    gaf = GramianAngularField(image_size=24, method='difference')
    noise = np.random.normal(0, 0.01, sample.shape)
    noisy_sample = sample + noise
    channels = []
    noisy_sample = noisy_sample.T  # Shape: (24, 3)
    for i in range(3):
        series = noisy_sample[:, i].reshape(1, -1)
        image = gaf.fit_transform(series)[0]
        channels.append(image)
    return np.stack(channels, axis=0)  # Shape: (3, 24, 24)

# Apply GAF with augmentation
X_train_gaf = np.array([transform_to_gaf(x) for x in X_train_scaled])
X_val_gaf = np.array([transform_to_gaf(x) for x in X_val_scaled])
X_test_gaf = np.array([transform_to_gaf(x) for x in X_test_scaled])

# Convert to tensors
X_train_tensor = torch.tensor(X_train_gaf, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val_gaf, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_gaf, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2D CNN Architecture
class GAF_CNN(nn.Module):
    def __init__(self):
        super(GAF_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.attention = nn.Sequential(
            nn.Conv2d(128, 1, kernel_size=1),
            nn.Sigmoid()
        )
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 4)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        attention = self.attention(x)
        x = x * attention
        x = x.view(x.size(0), -1)
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.fc2(x)

model = GAF_CNN().to(device)

# Advanced Training Configuration
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=30, steps_per_epoch=len(train_loader))

# Training Loop 
def train_model(model, train_loader, val_loader, epochs=30):
    best_acc = 0.0
    history = {'train_loss': [], 'val_acc': []}
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = 100 * correct / total
        scheduler.step()
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_gaf_cnn.pth')
        history['train_loss'].append(running_loss / len(train_loader))
        history['val_acc'].append(val_acc)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {history['train_loss'][-1]:.4f} | Val Acc: {val_acc:.2f}%")
    return history

print("\n=== Training 2D CNN ===")
history = train_model(model, train_loader, val_loader)

# Evaluation
model.load_state_dict(torch.load('best_gaf_cnn.pth'))
model.eval()

all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

test_acc = accuracy_score(all_labels, all_preds) * 100
print("\n=== Test Results ===")
print(f"Test Accuracy: {test_acc:.2f}%")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, 
                            target_names=['winter', 'spring', 'summer', 'autumn']))

# Confusion Matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['winter', 'spring', 'summer', 'autumn'],
            yticklabels=['winter', 'spring', 'summer', 'autumn'])
plt.title("Confusion Matrix (2D CNN with GAF)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Training Curves
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history['train_loss'], label='Training Loss')
plt.title("Training Loss Curve (2D CNN)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title("Validation Accuracy Curve (2D CNN)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.tight_layout()
plt.show()

# Model Comparison
print("\n=== Model Comparison ===")
print(f"MLP Test Accuracy: {mlp_test_acc:.2f}% (from Task 2)")
print(f"1D CNN Accuracy: {cnn_test_acc_1d:.2f}% (from Task 3)")
print(f"2D CNN with GAF Accuracy: {test_acc:.2f}%")
print(f"Improvement over MLP: {test_acc - mlp_test_acc:.2f}% points")
print(f"Improvement over 1D CNN: {test_acc - cnn_test_acc_1d:.2f}% points")
